# Import Required Libraries
Import necessary libraries such as numpy, tensorflow/keras, and matplotlib for visualization.

In [1]:
import pefile

import sys
import os
print(sys.version)
print(sys.executable)
sys.path.append('../')  # To import from parent directory
sys.path.append('../build/Debug')
sys.path.append(os.path.abspath("../build/Debug"))

pe = pefile.PE('../build/Debug/deep_learning_cuda.cp310-win_amd64.pyd')
for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print(entry.dll.decode())

import deep_learning_cuda

3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]
c:\Users\Damia\anaconda3\envs\my_cuda_env\python.exe
python310.dll
cudart64_110.dll
MSVCP140D.dll
VCRUNTIME140D.dll
VCRUNTIME140_1D.dll
ucrtbased.dll
KERNEL32.dll


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


# Load the MNIST Dataset
Load the MNIST dataset using Keras datasets API and split into training and test sets.

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(f"Training samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

Training samples: 60000, Test samples: 10000


# Preprocess the Data
Normalize the image data and convert labels to categorical format suitable for classification.

In [4]:
# Normalize images to [0, 1] and flatten
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

# Convert labels to categorical (one-hot)
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Instantiate the MLP Model
Define and compile a Multi-Layer Perceptron (MLP) model using your custom implementation.

In [5]:
from models.mlp_cuda import MLPCuda
from algorithms.sgd import SGD
from loss_functions.cross_entropy import cross_entropy_loss, cross_entropy_grad

# Define model dimensions
input_dim = 28*28
hidden_dim = 128
output_dim = 10

# Instantiate model and optimizer
mlp = MLPCuda(input_dim, hidden_dim, output_dim)
optimizer = SGD(mlp, lr=0.1)

# Train the Model
Train the MLP model on the training data and monitor training metrics.

In [6]:
# Train the model
optimizer.train(X_train, y_train_cat, cross_entropy_loss, cross_entropy_grad, epochs=10, batch_size=128, verbose=True)

Epoch 1/10, Loss: 2.1981
  Forward pass time: 0.9360 seconds
  Backward pass time: 0.7737 seconds
  Model Forward Measurements:
fc1 - time: 0.6235597133636475
relu - time: 0.0020008087158203125
fc2 - time: 0.27718567848205566
softmax - time: 0.02646470069885254
  Model Backward Measurements:
fc1 - time: 0.266345739364624
relu - time: 0.006409406661987305
fc2 - time: 0.014993429183959961
softmax - time: 0.4636268615722656
  Loss Forward Measurements:
cross_entropy - time: 0.019410133361816406
  Loss Backward Measurements:
cross_entropy - time: 0.009279489517211914
Epoch 2/10, Loss: 1.1273
  Forward pass time: 0.7533 seconds
  Backward pass time: 0.8616 seconds
  Model Forward Measurements:
fc1 - time: 0.6030740737915039
relu - time: 0.002820253372192383
fc2 - time: 0.1316542625427246
softmax - time: 0.007472991943359375
  Model Backward Measurements:
fc1 - time: 0.3766443729400635
relu - time: 0.011451482772827148
fc2 - time: 0.027500629425048828
softmax - time: 0.43796300888061523
  Lo

# Validate Model Performance on Test Set
Evaluate the trained model on the test set and display accuracy and loss metrics.

In [7]:
# Evaluate on test set
pred_probs, _ = mlp.forward(X_test)
pred_labels = np.argmax(pred_probs, axis=1)
true_labels = np.argmax(y_test_cat, axis=1)
accuracy = np.mean(pred_labels == true_labels)
test_loss, _ = cross_entropy_loss(pred_probs, y_test_cat)
print(f"Test Accuracy: {accuracy*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

Test Accuracy: 92.65%
Test Loss: 0.2858
